In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import pytorch3d
from pytorch3d.io import IO
from urdfpy import URDF

In [ ]:
urdf_path = '/private/home/ssax/hab_spot_arm/urdf/hab_spot_arm.urdf'


In [ ]:
for link in robot.joints:
    print(link._name)

In [ ]:
import urdfpy
import trimesh
from pathlib import Path
import numpy as np
# Load the URDF file
robot = urdfpy.URDF.load(urdf_path)

# Create an empty Trimesh object to store the mesh
mesh = trimesh.Trimesh()

# # Iterate through all the visual elements in the URDF
# for link in robot.links:
#     for visual in link.visuals:
#         # Check if the visual element has a mesh
#         print(visual.geometry.__dict__)
#         if visual.geometry.type == 'mesh':
#             # Load the mesh from the file specified in the URDF
#             mesh_file = visual.geometry.filename
#             mesh_data = trimesh.load_mesh(mesh_file)
#             # Transform the mesh to the visual element's pose if needed
#             # For example, you might need to apply the link's transform
#             # to position the mesh correctly
#             mesh_data.apply_transform(visual.origin)

#             # Add the mesh to the combined mesh
#             mesh += mesh_data

# Iterate through all the visual elements in the URDF
# Compute visual trimesh forward kinematics
cfg = {}
for leg in ['fl', 'fr', 'hl', 'hr']:
    cfg[f'{leg}.hy'] = 1*np.pi/6
    cfg[f'{leg}.kn'] = -2*np.pi/6
     
visuals = robot.visual_trimesh_fk(cfg={
    'arm0.hr0': np.pi,
    'arm0.sh0': - np.pi,
    'arm0.sh1': - np.pi * 1 / 8,
    'arm0.el0': np.pi * 5 / 8,
    'arm0.wr0': np.pi * 2 / 8 + np.pi / 12,
    # 'arm0.wr1': np.pi,
    'arm0.f1x': -np.pi * 1 / 4 - np.pi / 12,
    **cfg 
})

# Initialize an empty list to hold the transformed meshes
meshes = []

# Iterate over each visual in the visuals dictionary
for mesh, transform in visuals.items():
    mesh.apply_transform(transform)
    meshes.append(mesh)

# Combine the meshes
print(len(meshes))
combined_mesh = trimesh.util.concatenate(meshes)

# Export the combined mesh to an OBJ file
output_obj_path = "output_robot.obj"
combined_mesh.export(output_obj_path, file_type="obj", include_texture=True)

print(f"OBJ file saved to {output_obj_path}")

# from PIL import Image

# # Define a factor by which to lighten the image
# lighten_factor = 100  # this value can be adjusted to your needs

# # Open the image file
# with Image.open('material_0.png') as img:
#     img = img.convert('RGBA')  # Ensure the image has an alpha channel
#     data = img.getdata()
    
#     # Create a new data list to hold the modified pixels
#     new_data = []
#     for item in data:
#         # Change all white (also shades of whites)
#         # pixels to be fully transparent
#         r, g, b, a = item
#         new_data.append((r + lighten_factor, g + lighten_factor, b + lighten_factor, 255))
        
#     # Update image data
#     img.putdata(new_data)
    
#     # Save new image
#     img.save('material_0.png')

combined_mesh.show()


In [ ]:
from pytorch3d.renderer import (
    TexturesAtlas,
    TexturesVertex,
    TexturesUV,
)
from pytorch3d.io import IO, load_obj, load_ply

out_path = 'output_robot.obj'
out_path = Path(out_path)
os.chdir(out_path.parent)
with open(Path(out_path), 'r') as f:
    mesh = IO().load_mesh(out_path, device='cpu')
    print(f"Loaded mesh with {len(mesh.faces_packed())} faces")

def convert_to_textureVertex(textures_uv: TexturesUV, meshes:pytorch3d.structures.Meshes) -> TexturesVertex:
    verts_colors_packed = torch.zeros_like(meshes.verts_packed())
    verts_colors_packed[meshes.faces_packed()] = textures_uv.faces_verts_textures_packed()  # (*)
    return TexturesVertex(pytorch3d.structures.packed_to_list(verts_colors_packed, meshes.num_verts_per_mesh()))

if isinstance(mesh.textures, TexturesUV):
    mesh.textures = convert_to_textureVertex(mesh.textures, mesh)

In [ ]:
import os
# os.chdir('assets/spot_mesh/')

# mesh = IO().load_mesh('assets/spot_mesh/output_robot.obj')
from pytorch3d.io import load_objs_as_meshes, load_obj
from pytorch3d.vis.plotly_vis import plot_scene
plot_scene(
    {'scene': {
        'mesh': mesh
        }
     }
)